In [3]:
import numpy as np
import pandas as pd

from scipy.stats import bernoulli

from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

In [4]:
def loglik(theta, X, y):
    return np.mean(bernoulli.logpmf(y, mu=X.dot(theta)))

In [6]:
def grad_loglik(theta, X, y):
#     n = y.shape[0]
    mu = 1./(1 + np.exp(-X.dot(theta)))
    return np.mean(X.T.dot(y - mu))

In [8]:
def hess_loglik(theta, X):
    mu = 1./(1 + np.exp(-X.dot(theta)))
    D = np.diag(mu*(1-mu))
    return X.T.dot(D.dot(X))

If we're doing predictive deviance reltaive to $\beta^*$, the relevant term we need to estimate is approximately just $p$.

If its relative to $\beta_{full}$ when we're doing some kind of selection, should be $tr(I - S)$??

In [166]:
def mc_sim_term_star(theta, n=2000, niter=200):
    p = len(theta)
    ests = np.zeros(niter)
    
    for i in np.arange(niter):
        X = np.random.randn(n,p)
        mu = 1/(1 + np.exp(-X.dot(theta)))
        y = bernoulli.rvs(mu)
        y_star = bernoulli.rvs(mu)
        
        m = LogisticRegression(fit_intercept=False, penalty='none')
        m.fit(X,y)
        beta_hat = m.coef_
        
        a = beta_hat - theta
        b = X.T.dot(y_star - y)
        
        ests[i] = a.dot(b)
        
    return ests.mean()
    

In [167]:
p = 20
beta_0 = np.random.randn(p)
len(beta_0)

20

In [168]:
est = mc_sim_term_star(beta_0)

In [169]:
est

-19.849475457712753

In [294]:
def est_dev_term_lasso(theta, n=3000, niter=500):
    p = len(theta)
    
    mc_ests = np.zeros(niter)
    math_ests = np.zeros(niter)
    s = np.zeros(niter)
    
    for i in np.arange(niter):
        X = np.random.randn(n,p)
        mu = 1/(1 + np.exp(-X.dot(theta)))
        y = bernoulli.rvs(mu)
        y_star = bernoulli.rvs(mu)
        
        fm = LogisticRegression(fit_intercept=False, penalty='none')
        fm.fit(X,y)
        beta_full = fm.coef_[0]
        
        sm = LogisticRegression(fit_intercept=False, penalty='l1', solver='liblinear', C=.1)
        sm.fit(X,y)
        beta_sel = sm.coef_[0]
        
        a = beta_full - beta_sel
        b = X.T.dot(y_star - y)
        
        E = beta_sel != 0
        XE = X[:, E]
        SE = np.diag(E)[E,:]
        W = np.diag(1./(1 + np.exp(-X.dot(beta_sel))))
        HE = np.linalg.inv(XE.T.dot(W.dot(XE))).dot(SE).dot(np.linalg.inv(X.T.dot(W.dot(X))).dot(beta_full))
        math_ests[i] = np.sum(np.diag(HE)) - p
        
        s[i] = E.sum() - p
        
        mc_ests[i] = a.dot(b)
        
    return mc_ests.mean(), math_ests.mean(), s.mean()
    

In [295]:
p = 20
beta_0 = np.random.randn(p)
beta_0[np.random.choice(p,int(3*p/4),replace=False)] = 0
len(beta_0)

20

In [296]:
mc_est_lasso, math_est_lasso, s_est_lasso = est_dev_term_lasso(beta_0, niter=100)

In [297]:
mc_est_lasso

-5.391789764376155

In [298]:
math_est_lasso

-19.999999842629578

In [299]:
s_est_lasso

-5.31

In [341]:
def est_dev_term_rl(theta, n=3000, niter=500):
    p = len(theta)
    
    mc_ests = np.zeros(niter)
    math_ests = np.zeros(niter)
    s = np.zeros(niter)
    
    for i in np.arange(niter):
        X = np.random.randn(n,p)
        mu = 1/(1 + np.exp(-X.dot(theta)))
        y = bernoulli.rvs(mu)
        y_star = bernoulli.rvs(mu)
        
        fm = LogisticRegression(fit_intercept=False, penalty='none')
        fm.fit(X,y)
        beta_full = fm.coef_[0]
        W_full = np.diag(1./(1 + np.exp(-X.dot(beta_full))))
        
        sm = LogisticRegression(fit_intercept=False, penalty='l1', solver='liblinear', C=.1)
        sm.fit(X,y)
        beta_sel = sm.coef_[0]
        
        E = beta_sel != 0
        XE = X[:, E]
        s[i] = E.sum()

        rm = LogisticRegression(fit_intercept=False, penalty='none')
        rm.fit(XE, y)
        beta_rl = np.zeros_like(beta_full)
        beta_rl[E] = rm.coef_[0]
        
        a = beta_full - beta_rl
        b = X.T.dot(y_star - y)
        mc_ests[i] = a.dot(b)
        
        SE = np.diag(E)[E,:]
        W_rl = np.diag(1./(1 + np.exp(-X.dot(beta_rl))))
        HE = np.linalg.inv(XE.T.dot(W_rl.dot(XE))).dot(SE).dot(np.linalg.inv(X.T.dot(W_full.dot(X))).dot(beta_full)) / np.diag(W_full) * np.diag(W_rl)
        math_ests[i] = np.sum(np.diag(HE)) - p
                
    return mc_ests.mean(), math_ests.mean(), s.mean()
    

In [342]:
p = 20
beta_0 = np.random.randn(p)
beta_0[np.random.choice(p,int(3*p/4),replace=False)] = 0
len(beta_0)

20

In [343]:
mc_est_rl, math_est_rl, s_rl = est_dev_term_rl(beta_0, niter=100)

ValueError: operands could not be broadcast together with shapes (16,) (3000,) 

In [344]:
%debug

> /var/folders/6f/_l6d5fqj12zg60thjgmbdc5r0000gn/T/ipykernel_20032/626618145.py(38)est_dev_term_rl()
     36         SE = np.diag(E)[E,:]
     37         W_rl = np.diag(1./(1 + np.exp(-X.dot(beta_rl))))
---> 38         HE = np.linalg.inv(XE.T.dot(W_rl.dot(XE))).dot(SE).dot(np.linalg.inv(X.T.dot(W_full.dot(X))).dot(beta_full)) / np.diag(W_full) * np.diag(W_rl)
     39         math_ests[i] = np.sum(np.diag(HE)) - p
     40 

ipdb> W_full.shape
(3000, 3000)
ipdb> beta_full.shape
(20,)
ipdb> quit


In [338]:
mc_est_rl

-0.7912955512491406

In [339]:
math_est_rl

-19.999999700914284

In [340]:
s_rl

13.31